# Multiple PDF Scraping
## Kevin Dharma Kusumah
tutorial provided by Pythonic Accountant

import libraries

In [1]:
import re

import pdfplumber
import pandas as pd
from collections import namedtuple

Creating Line

In [2]:
Line = namedtuple('Line', 'company_id company_name doctype reference currency voucher inv_date due_date open_amt_tc open_amt_bc current months1 months2 months3')

Getting the line in PDF

In [3]:
company_re = re.compile(r'(V\d+) (.*) Phone:')
line_re = re.compile(r'\d{2}/\d{2}/\d{4} \d{2}/\d{2}/\d{4}')

In [4]:
line_re.search('478 USD BFS030OB/000000057 11/03/2014 12/30/2014 263.52 263.52 0.00 0.00 0.00 263.52')

<re.Match object; span=(27, 48), match='11/03/2014 12/30/2014'>

Determined the logic

In [5]:
Line(*'companynumber companyname invoice 478 USD BFS030OB/000000057 11/03/2014 12/30/2014 263.52 263.52 0.00 0.00 0.00 263.52'.split())

Line(company_id='companynumber', company_name='companyname', doctype='invoice', reference='478', currency='USD', voucher='BFS030OB/000000057', inv_date='11/03/2014', due_date='12/30/2014', open_amt_tc='263.52', open_amt_bc='263.52', current='0.00', months1='0.00', months2='0.00', months3='263.52')

Load the file

In [6]:
file = 'Sample Report Pythonic.pdf'

In [7]:
lines = []
total_check = 0

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('\n'):
            comp = company_re.search(line)
            if comp:
                vend_no, vend_name = comp.group(1), comp.group(2)

            elif line.startswith('INVOICES'):
                doctype = 'INVOICE'

            elif line.startswith('CREDITNOTES'):
                doctype = 'CREDITNOTE'

            elif line_re.search(line):
                items = line.split()
                lines.append(Line(vend_no, vend_name, doctype, *items))
                
            elif line.startswith('Supplier total'):
                tot = float(line.split()[2].replace(',', ''))
                total_check += tot

In [8]:
lines[:5]

[Line(company_id='V50012', company_name='Company A LTD', doctype='CREDITNOTE', reference='478', currency='USD', voucher='BFS030OB/000000057', inv_date='11/03/2014', due_date='12/30/2014', open_amt_tc='263.52', open_amt_bc='263.52', current='0.00', months1='0.00', months2='0.00', months3='263.52'),
 Line(company_id='V50012', company_name='CARGO SAFETRAVEL LTD', doctype='INVOICE', reference='J1062628/1', currency='USD', voucher='BFS026/000001137', inv_date='03/30/2019', due_date='03/30/2019', open_amt_tc='-102.00', open_amt_bc='-102.00', current='0.00', months1='-102.00', months2='0.00', months3='0.00'),
 Line(company_id='V50012', company_name='CARGO SAFETRAVEL LTD', doctype='INVOICE', reference='J1062638/1', currency='USD', voucher='BFS026/000001136', inv_date='02/28/2019', due_date='03/15/2019', open_amt_tc='-204.00', open_amt_bc='-204.00', current='0.00', months1='-204.00', months2='0.00', months3='0.00'),
 Line(company_id='V50014', company_name='ChaseB REGISTER USDA', doctype='INVOIC

In [9]:
df = pd.DataFrame(lines)
df

,company_id,company_name,doctype,reference,currency,voucher,inv_date,due_date,open_amt_tc,open_amt_bc,current,months1,months2,months3
0,V50012,Company A LTD,CREDITNOTE,478,USD,BFS030OB/000000057,11/03/2014,12/30/2014,263.52,263.52,0.00,0.00,0.00,263.52
1,V50012,CARGO SAFETRAVEL LTD,INVOICE,J1062628/1,USD,BFS026/000001137,03/30/2019,03/30/2019,-102.00,-102.00,0.00,-102.00,0.00,0.00
2,V50012,CARGO SAFETRAVEL LTD,INVOICE,J1062638/1,USD,BFS026/000001136,02/28/2019,03/15/2019,-204.00,-204.00,0.00,-204.00,0.00,0.00
3,V50014,ChaseB REGISTER USDA,INVOICE,110-135633,USD,BFS026/000001064,02/05/2019,04/29/2019,"-4,665.00","-4,665.00","-4,665.00",0.00,0.00,0.00
4,V50015,METALS CHEMICALS GROUP LIMITED,INVOICE,SI191433,USD,BFS026/000000908,02/12/2019,03/30/2019,-198.00,-198.00,0.00,-198.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,V50097,AT&T GLOBAL NETWORK SERVICES (US) BV,INVOICE,47255724,USD,BFS026/000001019,03/15/2019,04/30/2019,"-1,137.14","-1,137.14","-1,137.14",0.00,0.00,0.00
127,V50098,PLASTIC COATINGS LTD,INVOICE,234560,USD,BFS026/000000393,01/30/2019,04/01/2019,-480.00,-480.00,-480.00,0.00,0.00,0.00
128,V50102,TECHNICAL METALS LTD,INVOICE,221233,USD,BFS026/000000994,02/26/2019,03/30/2019,-30.00,-30.00,0.00,-30.00,0.00,0.00
129,V50102,TECHNICAL METALS LTD,INVOICE,221234,USD,BFS026/000000995,02/26/2019,03/30/2019,-30.00,-30.00,0.00,-30.00,0.00,0.00


Correcting the Data Types

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_id    131 non-null    object
 1   company_name  131 non-null    object
 2   doctype       131 non-null    object
 3   reference     131 non-null    object
 4   currency      131 non-null    object
 5   voucher       131 non-null    object
 6   inv_date      131 non-null    object
 7   due_date      131 non-null    object
 8   open_amt_tc   131 non-null    object
 9   open_amt_bc   131 non-null    object
 10  current       131 non-null    object
 11  months1       131 non-null    object
 12  months2       131 non-null    object
 13  months3       131 non-null    object
dtypes: object(14)
memory usage: 14.5+ KB


In [11]:
df['inv_date'] = pd.to_datetime(df['inv_date'])
df['due_date'] = pd.to_datetime(df['due_date'])

In [12]:
for col in df.columns[-6:]:
    df[col] = df[col].map(lambda x: float(str(x).replace(',', '')))

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   company_id    131 non-null    object        
 1   company_name  131 non-null    object        
 2   doctype       131 non-null    object        
 3   reference     131 non-null    object        
 4   currency      131 non-null    object        
 5   voucher       131 non-null    object        
 6   inv_date      131 non-null    datetime64[ns]
 7   due_date      131 non-null    datetime64[ns]
 8   open_amt_tc   131 non-null    float64       
 9   open_amt_bc   131 non-null    float64       
 10  current       131 non-null    float64       
 11  months1       131 non-null    float64       
 12  months2       131 non-null    float64       
 13  months3       131 non-null    float64       
dtypes: datetime64[ns](2), float64(6), object(6)
memory usage: 14.5+ KB


Checking the values

In [15]:
print(df['open_amt_bc'].sum())
print(total_check)

-129580.73999999999
-129580.73999999999


import to csv

In [16]:
df.to_csv('output_invoices.csv', index = False)